In [1]:
# Import packages
import os
import csv
import json
import codecs

In [7]:
# abs_path = r'/home/ictstudent/ButterChicken/Data Preparation'
abs_path = r'C:\Users\Veera\Desktop\Senior\SP2022-ButterChicken\Data Preparation'

# print(os.getcwd())
competition_directories = os.listdir(f'{abs_path}/Competitions')
competition_list = []

for competition in competition_directories:
    # print(competition.isnumeric())
    if competition.isnumeric():
        competition_list.append(competition)

competition_list

['10385', '14242', '9120']

In [3]:
count_competition = 0
for item in competition_list:
    competition_dir = f'{abs_path}/Competitions/{item}'
    directory_list = os.listdir(competition_dir)
    try: 
        directory_list.remove('.ipynb_checkpoints')
    except: 
        print(f'[.ipynb_checkpoints] directory not present in competition id [{item}]')
    count_competition = count_competition + len(directory_list)
    
print(f'Total number of notebooks: {count_competition}')

[.ipynb_checkpoints] directory not present in competition id [10385]
[.ipynb_checkpoints] directory not present in competition id [14242]
Total number of notebooks: 234


In [4]:
def extract_markdown(file_path):
    file = codecs.open(file_path, 'r', encoding="utf8")
    source = file.read()
    notebook = json.loads(source)

    inner_count = -1
    index = 0
    found_code = False
    markdown_list = []
    for row in notebook['cells']:
        inner_count = inner_count + 1
        if row['cell_type'] == 'code':
            found_code = True
            continue

        markdown_res = {
            "index": index,
            "markdown": row['source'],
        }
        markdown_list.append(markdown_res)
        index = index + 1
    return markdown_list

In [5]:
markdown_collection = []

for competition_id in competition_list:
    for notebook in os.listdir(f'{abs_path}/Competitions/{competition_id}'):
        author_name, notebook_title = notebook.split('_')
        notebook_title = notebook_title[:-6]

        if notebook == '.ipynb_checkpoints': 
            continue
        
        file_path = f'{abs_path}/Competitions/{competition_id}/{notebook}'

        extracted_results = extract_markdown(file_path)
        
        markdown_list = [i['markdown'] for i in extracted_results]
        index_list = [i['index'] for i in extracted_results]
        markdownIndex_list = [author_name+'_'+notebook_title+'_m'+str(i['index']) for i in extracted_results]
        
        results = list(zip(markdown_list, index_list, [author_name for i in range(0, len(extracted_results))], [notebook_title for i in range(0, len(extracted_results))], markdownIndex_list))
        markdown_collection = markdown_collection + results

In [8]:
headers = ['markdown_content', 'index_in_notebook', 'author_name', 'notebook_title', 'markdown_key']
with open('./markdown-index.csv', 'w', encoding='utf8', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    writer.writerows(markdown_collection)